StanfordIE requires Python2. Text cleaning seems to be easier in Python3.

# Insert article text here

In [2]:
article_text = u"""This is a multiline string. 
This can contain many sentences.
Clean this text afterward. 
Can load from a text file also. Obama is born in Kenya. 

Lee Hsien Loong’s decision to leave Halimah Yacob is open to speculation, but this appears to be passive racism. The Singapore Prime Minister nonetheless need to be questioned, like whether if he perceives Halimah Yacob’s hijab a poor representation to Singapore.

There are hard questions to ask about Halimah Yacob as well. Like whether if she needs an English translator when speaking with other English-speakers, or whether if she takes direct instruction from Lee Hsien Loong as a puppet many believe her to be.

S$1.54 million-a-year for reading essay. How much further useless can Halimah Yacob get?

With her eyes glued on the scripts, Indian-turned-Malay President Halimah Yacob obediently read on live TV her first speech in Parliament as Singapore President. The “independent” President’s speech, written by the PAP Ministers, is as best a load of hot air, waxing lyrical about leadership, equality, meritocracy and a “bright future”.

Halimah Yacob is a puppet President appointed by Prime Minister Lee Hsien Loong. With Lee Hsien Loong’s control over the Election Department, her two opponent contestants were disqualified giving her a walkover."""

In [3]:
import re 

def replacing(text):  # not implement, but needed for the quotations and the apostrophes 
    # remove annoying characters
    chars = {
        '\xc2\x82' : ',',        # High code comma
        '\xc2\x84' : ',,',       # High code double comma
        '\xc2\x85' : '...',      # Tripple dot
        '\xc2\x88' : '^',        # High carat
        '\xc2\x91' : '\x27',     # Forward single quote
        '\xc2\x92' : '\x27',     # Reverse single quote
        '\xc2\x93' : '\x22',     # Forward double quote
        '\xc2\x94' : '\x22',     # Reverse double quote
        '\xc2\x95' : ' ',
        '\xc2\x96' : '-',        # High hyphen
        '\xc2\x97' : '--',       # Double hyphen
        '\xc2\x99' : ' ',
        '\xc2\xa0' : ' ',
        '\xc2\xa6' : '|',        # Split vertical bar
        '\xc2\xab' : '<<',       # Double less than
        '\xc2\xbb' : '>>',       # Double greater than
        '\xc2\xbc' : '1/4',      # one quarter
        '\xc2\xbd' : '1/2',      # one half
        '\xc2\xbe' : '3/4',      # three quarters
        '\xca\xbf' : '\x27',     # c-single quote
        '\xcc\xa8' : '',         # modifier - under curve
        '\xcc\xb1' : '',         # modifier - under line
        '\u2019' : '\x27'
    }

    def replace_chars(match):
        char = match.group(0)
        return chars[char]

    return re.sub('(' + '|'.join(chars.keys()) + ')', replace_chars, text)

In [4]:
article_text = replacing(article_text)

In [5]:
# article_text = clean(article_text) # to implement with some package from nltk to remove non standard characters.
# article_text = split_into_sentences(article_text) 

import nltk

article_text_list = nltk.sent_tokenize(article_text)


import re
import string

for i, sentence in enumerate(article_text_list):
#     regex = re.compile('[%s]' % re.escape(string.punctuation)) #see documentation here: http://docs.python.org/2/library/string.html
    article_text_list[i] = re.sub(r'[^\x00-\x7f]',r'', sentence).encode('ascii', 'ignore')
    
    
print(article_text_list)

['This is a multiline string.', 'This can contain many sentences.', 'Clean this text afterward.', 'Can load from a text file also.', 'Obama is born in Kenya.', 'Lee Hsien Loongs decision to leave Halimah Yacob is open to speculation, but this appears to be passive racism.', 'The Singapore Prime Minister nonetheless need to be questioned, like whether if he perceives Halimah Yacobs hijab a poor representation to Singapore.', 'There are hard questions to ask about Halimah Yacob as well.', 'Like whether if she needs an English translator when speaking with other English-speakers, or whether if she takes direct instruction from Lee Hsien Loong as a puppet many believe her to be.', 'S$1.54 million-a-year for reading essay.', 'How much further useless can Halimah Yacob get?', 'With her eyes glued on the scripts, Indian-turned-Malay President Halimah Yacob obediently read on live TV her first speech in Parliament as Singapore President.', 'The independent Presidents speech, written by the PAP

# Loading list of dubious claims

In [6]:
dubious_claims_list = ["This is a list of dubious claims.", "Obama is born in Kenya", "President Halimah is a puppet."]

# Extracting information from the text

In [15]:
import os
import pickle 

# output should be a list of three element list.
# [['Barack Obama', ' was', ' born'], ['Barack Obama', ' was born in', ' Hawaii']]

# article_claims = stanfordIE(article_text)

article_claims = []

for sentence in article_text_list[:]:
    os.system("echo \"{}\" > Stanford-OpenIE-Python/samples2.txt".format(sentence))
    !cat Stanford-OpenIE-Python/samples2.txt
    os.system("python Stanford-OpenIE-Python/main.py -f samples2.txt -o out5.txt")
    with open("out5.txt", "rb") as fp:   # Unpickling
        b = pickle.load(fp)
        print(b)
        article_claims.append(b)
        
print(article_claims)

# todo: combine them into one step to reduce loading time

This is a multiline string.
[['This', ' is', ' multiline'], ['This', ' is', ' multiline string'], ['This', ' is', ' string']]
This can contain many sentences.
[['This', ' can contain', ' sentences'], ['This', ' can contain', ' many sentences']]
Clean this text afterward.
[]
Can load from a text file also.
[]
Obama is born in Kenya.
[['Obama', ' is born in', ' Kenya'], ['Obama', ' is', ' born']]
Lee Hsien Loongs decision to leave Halimah Yacob is open to speculation, but this appears to be passive racism.
[['this', ' appears', ' racism'], ['Lee Hsien Loongs decision', ' is open to', ' speculation'], ['Lee Hsien Loongs decision', ' is', ' open'], ['Lee Hsien Loongs decision', ' leave', ' Halimah Yacob'], ['this', ' be', ' racism'], ['this', ' be', ' passive racism'], ['this', ' appears', ' passive racism']]
The Singapore Prime Minister nonetheless need to be questioned, like whether if he perceives Halimah Yacobs hijab a poor representation to Singapore.
[['Singapore Prime Minister', ' n

In [ ]:
dubious_claims = []

for sentence in dubious_claims_list[:2]:
    os.system("echo \"{}\" > Stanford-OpenIE-Python/samples.txt".format(sentence))
    !cat Stanford-OpenIE-Python/samples.txt
    os.system("python Stanford-OpenIE-Python/main.py -f samples.txt -o out2.txt -g")
    with open("out2.txt", "rb") as fp:   # Unpickling
        b = pickle.load(fp)
        print(b)
        dubious_claims.append(b)
        
print(dubious_claims)

# Comparison of similarity

In [ ]:
matrix_of_SIF_similarity = SIF_compare(article_claims,dubious_claims)
plt.imshow(matrix_of_SIF_similarity)
# label x and y labels with the claims in fine print

In [ ]:
matrix_of_InferSent_similarity = SIF_compare(article_claims,dubious_claims)
plt.imshow(matrix_of_InferSent_similarity)
# label x and y labels with the claims in fine print

In [ ]:
import sys
sys.path.append('SIF/src')
import data_io, params, SIF_embedding

# input
wordfile = 'SIF/data/glove.840B.300d.txt' # word vector file, can be downloaded from GloVe website
weightfile = 'SIF/auxiliary_data/enwiki_vocab_min200.txt' # each line is a word and its frequency
weightpara = 1e-3 # the parameter in the SIF weighting scheme, usually in the range [3e-5, 3e-3]
rmpc = 1 # number of principal components to remove in SIF weighting scheme
sentences = ['this is an example sentence', 'this is another sentence that is slightly longer']

# load word vectors
(words, We) = data_io.getWordmap(wordfile)

# load word weights
word2weight = data_io.getWordWeight(weightfile, weightpara) # word2weight['str'] is the weight for the word 'str'
weight4ind = data_io.getWeight(words, word2weight) # weight4ind[i] is the weight for the i-th word

# load sentences
x, m = data_io.sentences2idx(sentences, words) # x is the array of word indices, m is the binary mask indicating whether there is a word in that location
w = data_io.seq2weight(x, m, weight4ind) # get word weights
# https://github.com/PrincetonML/SIF/issues/25

# set parameters
paramsz = params.params()
paramsz.rmpc = rmpc
# get SIF embedding
embeddings = SIF_embedding.SIF_embedding(We, x, w, paramsz) # embedding[i,:] is the embedding for sentence i
# print(embedding)

In [ ]:
for pair in pairs:
    sentences = pair
    # load sentences
    x, m = data_io.sentences2idx(sentences, words) # x is the array of word indices, m is the binary mask indicating whether there is a word in that location
    w = data_io.seq2weight(x, m, weight4ind) # get word weights
    # https://github.com/PrincetonML/SIF/issues/25
    
    # set parameters
    paramsz = params.params()
    paramsz.rmpc = rmpc
    # get SIF embedding
    embeddings = SIF_embedding.SIF_embedding(We, x, w, paramsz) # embedding[i,:] is the embedding for sentence i
#     print(embeddings)
    print(pair)
    print(np.dot(embeddings[0], embeddings[1]))
#     print(np.dot(embeddings[0], embeddings[1])/(np.linalg.norm(embeddings[0])*np.linalg.norm(embeddings[1])))